In [ ]:
from jdaviz.configs.mosviz.helper import MosViz

---
**NOTE**

Currently, the 2D spectrum viewer does not correctly default to the right viewer axes values. To get the 2D spectrum to show, switch "y axis" to `wave`.

---

In [ ]:
mosviz = MosViz()
mosviz.app

The MOSViz parsers accepts lists of `Spectrum1D`, `SpectralCube`, and `CCDData` for 1D, 2D, and image data respectively. Alternatively, and the method used below, users can provide lists of file paths and MOSViz will internally attempt to parse them as their respective data type.

In [ ]:
import glob

spectra_1d = []
spectra_2d = []
images = []


for file_path in glob.glob("/Users/nmearl/Downloads/mosviz_nirspec_data_0.3/level3/*"):
    if 'x1d' in file_path:
        spectra_1d.append(file_path)
    elif 's2d' in file_path:
        spectra_2d.append(file_path)
        
        
for i in range(len(spectra_1d)):
    images.append("/Users/nmearl/Downloads/mosviz_nirspec_data_0.3/level3/mosviz_cutouts/227.fits")

mosviz.load_1d_spectra(spectra_1d)
mosviz.load_2d_spectra(spectra_2d)
mosviz.load_images(images)

In [ ]:
from astropy.io import fits
from astropy.wcs import WCS
from spectral_cube import SpectralCube
import numpy as np
from glue.core.data import Data

def _parse_as_cube(path):
    with fits.open(path) as hdulist:
        data = hdulist[1].data

        if hdulist[1].header['NAXIS'] == 2:
            new_data = np.expand_dims(data, axis=1)
            hdulist[1].header['NAXIS'] = 3

        hdulist[1].header['NAXIS3'] = 1
        hdulist[1].header['BUNIT'] = 'dN/s'
        hdulist[1].header['CUNIT3'] = 'um'
        wcs = WCS(hdulist[1].header)
        
    from spectral_cube import BooleanArrayMask
    mask = BooleanArrayMask(mask=np.ones(new_data.shape).astype(bool), wcs=wcs) 

    return SpectralCube(new_data, wcs=wcs, mask=mask)#, mask=np.ones(new_data.shape).astype(bool), fill_value=False)

In [ ]:
cube = _parse_as_cube("/Users/nmearl/Downloads/mosviz_nirspec_data_0.3/level3/f170lp-g235m_mos_observation-6-c0e0_s00279_s2d.fits")

In [ ]:
cube.shape

In [ ]:
cube = cube.with_fill_value(0)

In [ ]:
cube[:, 0, :]